# LOAD DATA

In [1]:
import os
import json
import pandas as pd

DATA_FOLDER = os.getcwd() + '/review_scorer/data/'
SENTI_PATH = DATA_FOLDER + 'SentiWord_info.json'
DATA_PATH = DATA_FOLDER + 'data_origin.csv'
data: pd.DataFrame

with open(SENTI_PATH, mode='rt', encoding='UTF8') as f:
    senti = pd.DataFrame.from_dict(json.load(f))

data = pd.read_csv(DATA_PATH, encoding='UTF8')
data = data.dropna(axis=0)
data = data.sample(frac=1).reset_index(drop=True)

# TOKENIZER

In [2]:
from tqdm import tqdm
from twkorean import TwitterKoreanProcessor

processor = TwitterKoreanProcessor()
tokenize = processor.tokenize_to_strings
tokens = [tokenize(_) for _ in tqdm(data.review)]

100%|████████████████████████████████████| 71904/71904 [09:05<00:00, 131.86it/s]


# SET REVIEW SCORER

In [ ]:
from review_scorer import ReviewScorer

# Review scorer needs tokens of datas to train when initializing it.
# 리뷰 채점기 클래스를 생성할 때, 토크나이즈 된 데이터를 인자로 주어야 합니다.
rs = ReviewScorer(tokens=tokens, senti_dict_path=SENTI_PATH)

## TAGGING RIVIEW SCORER's SENTIMENTAL DICTIONARY

In [10]:
# Tagging review scorer's sentimental dictionary by category.
# 카테고리에 따라 리뷰 채점기의 감성사전을 태깅합니다.
rs.tag_senti_dict(categories={'taste': ['맛', '맛있다', '맛없다'],
                              'price': ['가격', '싸다', '비싸다', '저렴'],
                              'service': ['서비스', '친절', '싸가지'],
                              'atmosphere': ['인테리어', '분위기']},
                  topn=500)

# SCORING WITH REVIEW SCORER

In [11]:
start = 50000
for i in range(start, start + 10):
    print('index: ', i)
    print(rs.score_review(tokenize(data.review.iloc[i])))
    print(data.review.iloc[i])
    print()

index:  50000
{'taste': 2, 'price': 2, 'service': 0, 'atmosphere': 0}
칼국수 꼬들꼬들 맛있다. 만두도 괘안코~

index:  50001
{'taste': 9, 'price': 6, 'service': 0, 'atmosphere': 0}
분위기부터 맛, 서비스까지 모두 만족스러웠던 로바다야키!모두들 베스트로 꼽은 건 닭껍질!아직도 그 바삭함과 쫄깃함이 생각날 정도ㅠㅠ바로 구워서 주는 닭껍질은 정말...말을 잇지 못한다개인적으로 또다른 베스트는 닭 안창살!쫄깃한 식감이 아주 맘에 들었다닭가슴연골은 오독오독한 식감이 매력적이었다.재방문하면 이렇게 세가지는 꼭 먹을듯!!모듬채소구이도 고기만 먹기엔 죄스러운 마음을 달래주기 딱 좋고ㅎㅎ재료 식감도 하나하나 살아있어서 더 좋았다. 가지는 특이하게 양념해서 가쓰오부시까지 올라가 있었다 굿!양배추조차 왜이리 맛있는지..미소로 양념한 메로구이도 맛있었다! 메로구이니까 맛있을 수밖에ㅎㅎㅎ퀄리티가 떨어진다고 서비스로 주신 랍스터꼬리마저 괜찮았다!재방문한다면 우니타코였나..우니와 낙지를 섞은 요리와 브리치즈구이도 도전해보고싶다!!대부분 다찌형식?이지만 다행히! 테이블이 있어서 좋은 사람들과 즐거운시간을 보낼 수 있어 더욱 좋았던 밋업;)다만 넷까지는 괜찮지만, 다섯명이 앉는다면 한명의 희생이 필요한 자리ㅠㅠ재방문의사는 완전 있음!! 조용한 분위기라 두명이서 오붓하게 와서 사케 한잔씩 해도 정말정말 좋을 것 같다

index:  50002
{'taste': 0, 'price': 0, 'service': 0, 'atmosphere': 0}
Simple, vibrant, tastes like a simple seaside restaurant in Spain

index:  50003
{'taste': 11, 'price': 6, 'service': 2, 'atmosphere': 4}
식당 분위기도 딱 일본에 있는 느낌! 외관에서 굉장히 멋드러진 구 가옥같은 느낌을 받았고 인스타 사